# 安装包

In [1]:
!pip install onnxruntime 
!pip install cryptography

     |████████████████████████████████| 4.9 MB 27.9 MB/s 
     |████████████████████████████████| 3.6 MB 21.8 MB/s 


# 样例模型下载

In [2]:
!wget https://github.com/onnx/models/blob/main/vision/classification/resnet/model/resnet50-v1-12.onnx?raw=true -O resnet50-v1-12.onnx

--2022-03-16 05:27:48--  https://github.com/onnx/models/blob/main/vision/classification/resnet/model/resnet50-v1-12.onnx?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/onnx/models/raw/main/vision/classification/resnet/model/resnet50-v1-12.onnx [following]
--2022-03-16 05:27:48--  https://github.com/onnx/models/raw/main/vision/classification/resnet/model/resnet50-v1-12.onnx
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/onnx/models/main/vision/classification/resnet/model/resnet50-v1-12.onnx [following]
--2022-03-16 05:27:48--  https://raw.githubusercontent.com/onnx/models/main/vision/classification/resnet/model/resnet50-v1-12.onnx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.19

# 确认读取ONNX模型

In [3]:
import cv2 as cv
import onnxruntime

model_path = 'resnet50-v1-12.onnx'

onnx_session = onnxruntime.InferenceSession(
    model_path,
    providers=['CPUExecutionProvider'],
)

print(onnx_session.get_inputs()[0])
print(onnx_session.get_outputs()[0])

NodeArg(name='data', type='tensor(float)', shape=[1, 3, 224, 224])
NodeArg(name='resnetv17_dense0_fwd', type='tensor(float)', shape=[1, 1000])


# 通过密钥生成方式加密/解密

In [4]:
from cryptography.fernet import Fernet
from cryptography.fernet import InvalidToken

# 密钥生成
key = Fernet.generate_key()

# 密钥保存
with open('./key.txt', 'wb') as f:
    f.write(key)
print(key)

b'PZz9_vQzHQtJvfTFv_F5iTIiWs7oZwSPs8A6rtapC0s='


In [5]:
# 以二进制模式加载ONNX模型
onnx_data = None
with open('resnet50-v1-12.onnx', 'rb') as onnx_file:
    onnx_data = onnx_file.read()

# 生成具有指定键的Fernet对象
fernet = Fernet(key)

# 加密
encrypt_data = fernet.encrypt(onnx_data)

# 将加密数据写入文件
with open('resnet50-v1-12.dat', 'wb') as f:
    f.write(encrypt_data)

In [6]:
# 读取密钥
with open('key.txt', 'rb') as f:
    key = f.read()
print(key)

# 读取加密数据
with open('resnet50-v1-12.dat', 'rb') as f:
    encryp_data = f.read()

# 生成具有指定密钥的Fernet对象
fernet = Fernet(key)

# 解密
try:
    decrypt_data = fernet.decrypt(encryp_data)
except InvalidToken:
    print('InvalidToken')

b'PZz9_vQzHQtJvfTFv_F5iTIiWs7oZwSPs8A6rtapC0s='


In [7]:
# 加载模型
import cv2 as cv
import onnxruntime

# 加载解密后的ONNX模型
onnx_session = onnxruntime.InferenceSession(
    decrypt_data,
    providers=['CPUExecutionProvider'],
)

print(onnx_session.get_inputs()[0])
print(onnx_session.get_outputs()[0])

NodeArg(name='data', type='tensor(float)', shape=[1, 3, 224, 224])
NodeArg(name='resnetv17_dense0_fwd', type='tensor(float)', shape=[1, 1000])


# 通过指定密码进行加密/解密

In [8]:
import base64
from cryptography.fernet import Fernet
from cryptography.fernet import InvalidToken
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

# 以二进制模式加载 ONNX 模型
with open('resnet50-v1-12.onnx', 'rb') as onnx_file:
    onnx_data = onnx_file.read()

# 密码
password_text = 'P@ssw0rd'

# 使用Salt实例化
# Salt应该为每个用户生成一个随机值，但是在示例中我们暂时使用一个固定值
salt = b'salt'  
stretching_iterations = 390000
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=stretching_iterations,
)

# 密钥生成
password = password_text.encode("utf-8")
key = base64.urlsafe_b64encode(kdf.derive(password))

# 生成具有指定密钥的 Fernet 对象
fernet = Fernet(key)

# 加密
encrypt_data = fernet.encrypt(onnx_data)

# 将加密数据写入文件
with open('resnet50-v1-12-password.dat', 'wb') as f:
    f.write(encrypt_data)

In [9]:
# 读取加密数据
with open('resnet50-v1-12-password.dat', 'rb') as f:
    encryp_data = f.read()

# 密码
password_text = 'P@ssw0rd'

# 使用Salt实例化
# Salt应该为每个用户生成一个随机值，但是在示例中我们暂时使用一个固定值
salt = b'salt'  
stretching_iterations = 390000
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=stretching_iterations,
)

# 生成密钥
password = password_text.encode("utf-8")
key = base64.urlsafe_b64encode(kdf.derive(password))

# 生成具有指定密钥的 Fernet 对象
fernet = Fernet(key)

# 解密
try:
    decrypt_data = fernet.decrypt(encryp_data)
except InvalidToken:
    print('InvalidToken')

In [10]:
# 加载模型
import cv2 as cv
import onnxruntime

# 加载解密后的onnx模型
onnx_session = onnxruntime.InferenceSession(
    decrypt_data,
    providers=['CPUExecutionProvider'],
)

print(onnx_session.get_inputs()[0])
print(onnx_session.get_outputs()[0])

NodeArg(name='data', type='tensor(float)', shape=[1, 3, 224, 224])
NodeArg(name='resnetv17_dense0_fwd', type='tensor(float)', shape=[1, 1000])
